In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Define path to the directory in Google Drive
drive_path = '/content/drive/My Drive/Churn_Model/'

# Load model
model = load_model(drive_path + 'model.h5')

# Load label encoder
with open(drive_path + 'label_encoder_gender.pkl', 'rb') as f:
    label_encoder_gender = pickle.load(f)

# Load scaler
with open(drive_path + 'scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

# Load one-hot encoder
with open(drive_path + 'onehot_encoder.pkl', 'rb') as f:
    onehot_encoder_geo = pickle.load(f)

In [4]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [5]:
## One-hot encode the Geography
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [6]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [7]:
## encode categorical variables
input_df["Gender"] = label_encoder_gender.transform(input_df["Gender"])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [8]:
## concatination one hot encoded
input_df = pd.concat([input_df.drop("Geography", axis=1), geo_encoded_df], axis=1)

In [9]:
## Scaling the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [10]:
## Predict churn
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step


array([[0.04099528]], dtype=float32)

In [11]:
prediction_proba = prediction[0][0]

In [12]:
prediction_proba

np.float32(0.040995277)

In [13]:
if prediction_proba > 0.5:
    print("The customer is likely to churn with a probability of {:.2f}%".format(prediction_proba * 100))
else:
    print("The customer is unlikely to churn with a probability of {:.2f}%".format((1 - prediction_proba) * 100))

The customer is unlikely to churn with a probability of 95.90%
